## Caida Normal Caida Fuerte (CNCF)
##### Los precios caigan y toquen PM40, antes deben de haber caido 3 - 5 puntos (Caida Normal)
##### si los precios caen mas de 5 puntos (Caida fuerte)
##### Vela martillo indica que esta parando la caida
##### Trazo una linea de tendencia siguiendo la caida, entra cuando rompe la tendencia

In [1]:
import pandas as pd
import numpy as np
#import time
#import matplotlib.pyplot as plt
import os
import talib as ta
from scipy import stats
from scipy.signal import argrelextrema
from datetime import date, datetime
#from dateutil.relativedelta import relativedelta
from bokeh.plotting import figure, show, column
from sklearn.linear_model import LinearRegression
from backtesting import Backtest, Strategy

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
df_h=pd.read_csv('data/dataxh.txt',sep='\t')
tickers = [
    'SPY',
    'META',
    'AAPL',
    'AMZN',
    'NFLX',
    'MRNA',
    'TSLA',
    'TNA',
    'GLD',
    'SLV',
    'USO',
    'BAC',
    'CVX',
    'XOM',
    'QQQ',
    'MSFT',
    'NVDA',
    'WMT',
    'BA',
    'DIS',
    'CAT',
    'IBM',
    'WFC',
    'PLTR',
    'AMD',
    'AVGO',
    'HOOD',
    'CRWV',
    'MSTR',
    'UNH',
    'GOOG',
    'APP',
    'UBER',
]

#    'SPY',
#    'META',
#    'AAPL',
#    'AMZN',
#    'NFLX',
#    'MRNA',
#    'TSLA',
#    'TNA',
#    'GLD',
#    'SLV',
#    'USO',
#    'BAC',
#    'CVX',
#    'XOM',
#    'QQQ'

In [3]:
df = pd.DataFrame()
for ticker in tickers:
    company = df_h.query("companyName==@ticker").copy()
    company.sort_values(by=['datetime'])
    #company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    #company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    #company['datetime'] = pd.to_datetime (company.index)
    #company['companyName'] = ticker
    company['SMA20'] = company['Close'].rolling(20).mean()
    company.dropna(inplace=False)
    company['SMA40'] = company['Close'].rolling(40).mean()
    company.dropna(inplace=False)

    #EVALUACION DE VELAS ALCISTAS
    company["CDLHAMMER"] = ta.CDLHAMMER(company['Open'], company['High'], company['Low'], company['Close'])
    company["CDLINVERTEDHAMMER"] = ta.CDLINVERTEDHAMMER(company['Open'], company['High'], company['Low'], company['Close']) 
    company["CDLMORNINGSTAR"] = ta.CDLMORNINGSTAR(company['Open'], company['High'], company['Low'], company['Close'])
    
    company["CDLHARAMI"] = ta.CDLHARAMI(company['Open'], company['High'], company['Low'], company['Close'])   
    company["CDLENGULFING"] = ta.CDLENGULFING(company['Open'], company['High'], company['Low'], company['Close']) 

    company["CDLMORNINGDOJISTAR"] = ta.CDLMORNINGDOJISTAR(company['Open'], company['High'], company['Low'], company['Close']) 
    company["CDLTAKURI"] = ta.CDLTAKURI(company['Open'], company['High'], company['Low'], company['Close']) 
    company["CDLDRAGONFLYDOJI"] = ta.CDLDRAGONFLYDOJI(company['Open'], company['High'], company['Low'], company['Close']) 
    company["CDLMARUBOZU"] = ta.CDLMARUBOZU(company['Open'], company['High'], company['Low'], company['Close'])     

    company["CDL3BLACKCROWS"] = ta.CDL3BLACKCROWS(company['Open'], company['High'], company['Low'], company['Close'])
    company["CDL2CROWS"] = ta.CDL2CROWS(company['Open'], company['High'], company['Low'], company['Close'])
    company["CDLIDENTICAL3CROWS"] = ta.CDLIDENTICAL3CROWS(company['Open'], company['High'], company['Low'], company['Close'])

    company["CDL3WHITESOLDIERS"] = ta.CDL3WHITESOLDIERS(company['Open'], company['High'], company['Low'], company['Close'])



    #df["cdlpiercing"] = ta.CDLPIERCING(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    #df["cdlclosingmarubozu"] = ta.CDLCLOSINGMARUBOZU(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    #df["cdlmarubozu"] = ta.CDLMARUBOZU(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    #df["cdl3whitesoldiers"] = ta.CDL3WHITESOLDIERS(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    #df["cdlharami"] = ta.CDLHARAMI(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    #df["cdlharamicross"] = ta.CDLHARAMICROSS(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])    
    #df["cdlladderbottom"] = ta.CDLLADDERBOTTOM(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])

     
    
    df = pd.concat([df, company],ignore_index=True)

In [4]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [5]:
def tipo_vela(df):
    if (df['CDLHAMMER']>0):
        return 'Hammer'
    elif (df['CDLINVERTEDHAMMER']>0):
        return 'InvHammer'
    elif (df['CDLMORNINGSTAR']>0):
        return 'MorningStar'    
    elif (df['CDLHARAMI']>0):
        return 'Harami'
    elif (df['CDLENGULFING']>0):
        return 'Engulfing'        
    elif (df['CDLMORNINGDOJISTAR']>0):
        return 'MorningDojiStar'
    elif (df['CDLTAKURI']>0):
        return 'Takuri '
    elif (df['CDLDRAGONFLYDOJI']>0):
        return 'DragonflyDoji'
    elif (df['CDLMARUBOZU']>0):
        return 'Marubozu'
    elif (df['CDL3BLACKCROWS']>0):
        return '3BlackCrows'
    elif (df['CDL2CROWS']>0):
        return '2Crows'
    elif (df['CDLIDENTICAL3CROWS']>0):
        return 'Identical3Crows'
    elif (df['CDL3WHITESOLDIERS']>0):
        return '3WhiteSoldiers'
df['tipo_vela'] = df.apply(tipo_vela, axis=1)


In [6]:
df = df.drop(['CDLHAMMER', 'CDLINVERTEDHAMMER','CDLMORNINGSTAR','CDLHARAMI','CDLENGULFING',
              'CDLMORNINGDOJISTAR','CDLTAKURI','CDLDRAGONFLYDOJI','CDLMARUBOZU','CDL3BLACKCROWS',
              'CDL2CROWS','CDLIDENTICAL3CROWS','CDL3WHITESOLDIERS'], axis=1)

In [7]:
df[df['tipo_vela'].isna()!=True]

,Close,High,Low,Open,Volume,datetime,companyName,SMA20,SMA40,tipo_vela
14,439.480011,439.660004,436.230011,436.329987,20473315,2023-06-15 13:30:00+00:00,SPY,NaN,NaN,Marubozu
16,440.920013,441.130096,440.059998,440.720001,10421756,2023-06-15 15:30:00+00:00,SPY,NaN,NaN,Takuri
29,435.881409,436.369995,435.029999,435.299988,10120204,2023-06-20 14:30:00+00:00,SPY,439.638348,NaN,Harami
30,437.200012,437.359985,435.850006,435.899994,6350887,2023-06-20 15:30:00+00:00,SPY,439.634325,NaN,Marubozu
39,436.679901,436.700012,435.010010,435.105011,6660049,2023-06-21 17:30:00+00:00,SPY,438.322371,438.211575,Engulfing
...,...,...,...,...,...,...,...,...,...,...
113552,87.310000,87.330000,87.090001,87.300000,8839,2025-06-09 17:00:00+00:00,UBER,86.493510,85.655830,Takuri
113569,86.590000,86.650000,86.390000,86.620000,2866491,2025-06-10 18:00:00+00:00,UBER,87.121690,86.691100,Takuri
113571,86.900000,87.200000,86.050000,86.050000,10259,2025-06-11 04:00:00+00:00,UBER,87.065940,86.741350,Engulfing
113575,86.900000,87.290000,86.001200,86.146200,45759,2025-06-11 08:00:00+00:00,UBER,86.945995,86.849350,Engulfing


## Detect Pivots/Fractals

In [8]:
ord=10
#for ticker in tickers:
#print("====", ticker)
#dfpl = df[(df.companyName==ticker)] #df.query("companyName=='AAPL'")
max_idx = argrelextrema(df['Close'].values, np.greater, order=ord)[0]
min_idx = argrelextrema(df['Close'].values, np.less, order=ord)[0]
#print(max_idx)
#print(min_idx)
#df['pivotHigh'] = np.nan
#df['pivotLow'] = np.nan
#df['isPivot'] = np.nan
#df['isPivot'] = np.nan
# Aplicar el cálculo solo a los índices en la lista
df.loc[max_idx, 'pivotHigh'] = df['High']+1e-3
df.loc[min_idx, 'pivotLow'] = df['Low']-(1e-3)
df.loc[max_idx, 'isPivot'] = 1
df.loc[min_idx, 'isPivot'] = 2

In [9]:
#CODIGO DE REVISION CARLOS
#dfpl = df.iloc[500:970]
#id: 479
dfpl = df.iloc[404:554]

p = figure(width=2500, height=500,
        title="CNCF",
        background_fill_color="#efefef",
        tooltips=[("Index", "@index"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close")]
        )
p.xaxis.major_label_orientation = 0.8 # radians
p.x_range.range_padding = 0.05
#p.xaxis.axis_line_join = "bevel" # radians
p.xaxis.axis_line_width = 2

p.segment("index", "High", "index","Low",  color="black", line_width=1, source=dfpl)

inc = dfpl.query("Close>Open")
dec = dfpl.query("Open>Close")

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="red",
    line_color="red",    
    source=dec   
)


p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="green",
    line_color="green", 
    source=inc   
)


p.line(
    x="index", 
    y="SMA20", 
    color="#ffb81c",
    legend_label="SMA20",
    source=dfpl)

p.line(
    x="index", 
    y="SMA40", 
    color="red",
    legend_label="SMA40",
    source=dfpl)

p.scatter(x="index", y="pivotHigh", marker="circle", size=10,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista", source=dfpl[(dfpl.isPivot==1)])

p.scatter(x="index", y="pivotLow", marker="circle", size=10,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista", source=dfpl[(dfpl.isPivot==2)])

show(p)

In [10]:
diffs = []
for i, row in df.iterrows():
    pivotHigh = row['pivotHigh']
    valor_actual = row['pivotLow']
    
    #if pivotHigh!=np.nan:  # solo aplicar la lógica cuando tipo sea 'X'
    if valor_actual>0:  # solo aplicar la lógica cuando tipo sea 'X'      
        valor_previo = df.loc[:i - 1, 'pivotHigh'][::-1].dropna()
        if not valor_previo.empty:
            ultimo_valor = valor_previo.iloc[0]
        else:
            ultimo_valor = np.nan  # o np.nan
    
        dif = ultimo_valor - valor_actual 
       
    else:
        dif = np.nan # o 0, o dejar el valor original si ya existía        
    diffs.append(dif)
df['dif_pivot'] = diffs

In [11]:
#df['dif_ant1'] = df['Close'].shift(1) - df['pivotLow']
#df['dif_ant2'] = df['Close'].shift(2) - df['pivotLow']
df['dif_ant1'] = df['Close'].shift(1) - df['Close']
df['dif_ant2'] = df['Close'].shift(2) - df['Close']
df[df['dif_pivot'].notna()]

,Close,High,Low,Open,Volume,datetime,companyName,SMA20,SMA40,tipo_vela,pivotHigh,pivotLow,isPivot,dif_pivot,dif_ant1,dif_ant2
62,431.429993,432.850006,431.190002,432.640015,16895156,2023-06-26 19:30:00+00:00,SPY,433.863991,435.604056,None,NaN,431.189002,2.0,12.711991,1.209015,0.684998
102,437.350006,437.850006,437.059998,437.730011,9701525,2023-07-06 14:30:00+00:00,SPY,442.536256,439.333952,None,NaN,437.058998,2.0,7.072007,0.369995,5.779999
117,438.390015,439.079987,437.584991,438.709900,6454870,2023-07-10 15:30:00+00:00,SPY,440.075380,440.898018,None,NaN,437.583991,2.0,5.057023,0.329987,0.879883
176,451.519897,452.769989,451.440002,452.114990,9510899,2023-07-20 18:30:00+00:00,SPY,453.960474,451.886115,None,NaN,451.439002,2.0,4.901994,0.570099,2.140503
212,452.464996,453.140015,451.549988,452.739990,15556788,2023-07-27 19:30:00+00:00,SPY,455.286150,454.366756,None,NaN,451.548988,2.0,6.862016,0.274994,1.470001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113442,87.510002,87.833504,86.699997,87.205002,3941872,2025-05-23 13:30:00+00:00,UBER,89.496975,90.609268,None,NaN,86.698997,2.0,6.902002,0.459999,0.829994
113472,83.279999,83.879997,83.220001,83.779999,1484136,2025-05-30 15:30:00+00:00,UBER,86.477850,87.435525,None,NaN,83.219001,2.0,6.661996,0.520004,0.599998
113490,82.540001,82.699997,82.334999,82.620003,2465686,2025-06-03 19:30:00+00:00,UBER,83.475140,85.263620,None,NaN,82.333999,2.0,2.034303,0.080002,0.222801
113515,83.385000,83.970000,83.080000,83.925000,1675601,2025-06-05 12:00:00+00:00,UBER,83.728460,83.485925,None,NaN,83.079000,2.0,1.632000,0.535000,0.725000


##### Casos
###### Precio esta por debajo o cerca a PM40 y ademas haya caido mas de 3 puntos
###### Caida normal entre 3-5 puntos
###### Caida fuerte mas de 5 puntos

In [12]:
def cncf01(df):
    if ((df['isPivot']==2) & (df['dif_ant1']>=3) & (df['dif_ant1']<=5) & (df['SMA40']>df['SMA20'])):
        return 1
    elif ((df['isPivot']==2) & (df['dif_ant1']>5) & (df['SMA40']>df['SMA20'])):
        return 2
    else:
        return 0
df['cncf01'] = df.apply(cncf01, axis=1)

#### Detect price channel

In [13]:
def collect_channel(candle, backevalTrend, trendH): 
    localdf = dfpl.loc[candle-backevalTrend:candle] #tomar en cuenta el backcandles
    highs = localdf.loc[localdf.index.isin(trendH)].High.values
    idxhighs = localdf.loc[localdf.index.isin(trendH)].High.index
    if len(highs)>=2:       
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)       
        return(sl_highs, interc_highs, r_value_h**2)
    else:
        return(0,0,0)    

In [14]:
def backcandle(candle):
    
    trendH = []
    backeval = dfpl.index[0]
    val = 0
    
    dfpl['PM40_PM20'] = dfpl['SMA40']-dfpl['SMA20']

    if (dfpl.loc[candle,"PM40_PM20"]>0):
        idx_negativo = dfpl.loc[:candle, 'PM40_PM20'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4
        val = candle - backeval
    
    if val>75:
        backeval=candle-75

    #print ("backeval:", backeval)
        
    if (dfpl.loc[backeval,"PM40_PM20"]>0):
        idx_negativo = dfpl.loc[:backeval, 'PM40_PM20'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4

    if val>75:
        backeval=candle-75

    #ini = candle - backeval
    ini = backeval
    trendprev=dfpl.loc[ini:candle]
    trend = trendprev.reset_index(drop=False)

    idxhighs = trend.index.to_numpy().reshape(-1,1) # Convertir X a una matriz 2D
    highs = trend.High.values
    modelo1 = LinearRegression()
    modelo1.fit(idxhighs, highs)
    # Obtener la predicción de la línea de regresión
    Y_pred = modelo1.predict(idxhighs)
    # Calcular la distancia solo de los puntos que están POR ENCIMA de la línea
    trend['Distancia'] = trend["High"] - Y_pred  # Diferencia entre el valor real y la predicción
    # Filtrar los puntos que están por encima (donde Y > Y_pred, es decir, Distancia > 0)
    df_encima = trend[trend['Distancia'] > 0]
    puntos_mas_lejanos = df_encima.nlargest(3, 'Distancia')   #2 puntos mas lejanos
    trendH = puntos_mas_lejanos["index"].tolist()
    trendH.append(candle)   
    
    return trendH, backeval

### Detect break out

In [15]:
def trendChannel(row,candle, backeval, window, sl_highs, interc_highs):
    if (candle-backeval-window)<0:
        return np.nan
    ini=candle-backeval
    fin=candle+window
    trendcurrhigh = np.where(np.logical_or((row.index > fin),  (row.index <  ini)), np.nan, (sl_highs*row.index + interc_highs))
    return trendcurrhigh

In [16]:
def isBreakOut(candleEval,backcandles, window, start, candle):
    if (candleEval-backcandles-window)<0:
        return 0

    if candleEval==start:
        prev_idx = candleEval
    else:
        prev_idx = candleEval-1
        
    prev_high = dfpl.loc[prev_idx].High
    prev_low = dfpl.loc[prev_idx].Low
    prev_close = df.loc[prev_idx].Close
    
    curr_idx = candleEval
    curr_high = dfpl.loc[candleEval].High
    curr_low = dfpl.loc[candleEval].Low
    curr_close = dfpl.loc[candleEval].Close
    curr_open = dfpl.loc[candleEval].Open

    #trend_prevlow=dfpl.loc[prev_idx].trendcurrlow
    trend_prevhigh=dfpl.loc[prev_idx].trendcurrhigh

    #trend_currlow=dfpl.loc[candleEval].trendcurrlow
    trend_currhigh=dfpl.loc[candleEval].trendcurrhigh

    if ( prev_low < trend_prevhigh and
        prev_close > trend_prevhigh and
        curr_open > trend_currhigh and
        curr_close > trend_currhigh and
        candleEval>candle): #and r_sq_h > 0.9
        return 2

    elif (curr_open > trend_currhigh and
        curr_close > trend_currhigh and candleEval>candle):
        return 3
    
    else:
        return np.nan

In [17]:
def breakpointpos(x):
    if x['isBreakOutIni'] in (2,3):
        return x['Low']+3e-3
    elif x['isBreakOutIni']==1:
        return x['High']-3e-3
    else:
        return np.nan

In [18]:
## Evaluando casos
df_casos = pd.DataFrame()
caso = 0
ticker2 = ""
window=5
backeval=75
backevalTrend=0
dfpl = pd.DataFrame
trendH = []
trendL = []
cant = 0
for i, row in df.iterrows():
    if(df['cncf01'][i]>=1): #posibles casos 
        print("posible caso:",i, df['companyName'][i])
        cant = 0
        candle = i
        cnt = 0
        valiniHigh = df.loc[candle,"High"]
        #valiniLow = df.loc[candle,"Low"]
        ticker = df['companyName'][i]
        #Reinicio de casos por company
        if ticker2 != ticker:
            ticker2 = ticker
            caso = 0
        dfpl = df[(df.companyName==ticker)].loc[i-backeval:i+backeval].copy()
        #print("rowMin:", dfpl.index[0],", rowMax:", dfpl.index[-1])
                   
        h=1
        l=1

        cant = cant +1
        cantHorasTrend=0
        #print("opcionH:",h,"opcionL:",l)
        trendH, backevalTrend = backcandle(candle)
        #print ("-->trendH:",trendH)
        #print ("-->backevalTrend:",backevalTrend)    
        
        cantHorasTrend = (dfpl.loc[candle,"datetime"] - dfpl.loc[backevalTrend,"datetime"]).total_seconds() / 3600

        if (cantHorasTrend>48): #Verificacion de caida por lo menos 2 dias
           backevalTrend2 = candle - backevalTrend
           #print ("backevalTrend2", backevalTrend2)
                       
           #revision de trend negativo indica que esta cayendo, solo me quedo con estos casos
           sl_highs, interc_highs, r_sq_h = collect_channel(candle, backevalTrend2, trendH)
                   
    
           dfpl.loc[trendH, "trendH"] = 1
        
           
           #print("sl_highs:", sl_highs, ", sl_lows:", sl_lows, ", r_sq_h:", r_sq_h)
           if (sl_highs<=0): #solo tendencias a la baja and ind_vela>0 sl_lows<0 and r_sq_h>=0.7  and sl_highs<=-0.1 and ind_vela>0
               
               #crear linea de tendencia high y low
               dfpl['trendcurrhigh']  = trendChannel(dfpl,candle,backevalTrend2,window,sl_highs,interc_highs)                
          
               promVol = dfpl.loc[i-backeval:i]["Volume"].mean()                
               
               dfpl["bearishSlope"] = np.nan #pendiente bajista en vacio
               dfpl.loc[i, "bearishSlope"] = 1 #pendiente bajista solo al punto evaluado            
               dfpl["ind_posicion"] = 0
               dfpl.loc[(dfpl.index < i), 'ind_posicion'] = -1
               dfpl.loc[(dfpl.index > i), 'ind_posicion'] = 1
               dfpl["promVol"] = promVol
               dfpl["sl_highs"] = sl_highs
               dfpl["r_sq_h"] = r_sq_h
        
               start = dfpl.index[0]
               dfpl["isBreakOut"] = [isBreakOut(candleEval, backevalTrend2, window, start, candle) for candleEval in dfpl.index]
               # Solo me quedo con el primer BREAK OUT
               cnt = dfpl.query("isBreakOut in (2,3) and Close>Open").shape[0] 
               id=0
               id2=0
               if cnt>0:
                   id = dfpl.query("isBreakOut in (2,3) and Close>Open").index[0]                        
                   #dfpl['isBreakOutIni'] = np.where(id!=dfpl.index, np.nan, dfpl['isBreakOut'])                        
                   dfpl.loc[id,'isBreakOutIni'] = 1
                   dfpl['breakpointpos'] = dfpl.apply(lambda row: breakpointpos(row), axis=1)
                   cnt2 = dfpl.query("index>@id and isPivot==1").shape[0]
                   
                   if cnt2>0:
                       id2 = dfpl.query("index>@id and isPivot==1").index[0]
                       dfpl.loc[id2,'isBreakOutFinal'] = 1
                   else:
                       dfpl['isBreakOutFinal'] = np.nan                      
                   
               else:
                   dfpl['isBreakOutIni'] = np.nan                        
                   dfpl['breakpointpos'] = np.nan
        
               if cnt>0:
                   #INSERT CASO
                   caso=caso+1                            
                   dfpl["caso"] = caso
                   #print(i, df['companyName'][i])
                   if len(df_casos)<1:
                       df_casos = dfpl
                   else:
                       df_casos = pd.concat([df_casos, dfpl],ignore_index=False)

posible caso: 1954 SPY
posible caso: 1997 SPY


posible caso: 2179 SPY
posible caso: 2529 SPY


posible caso: 2759 SPY
posible caso: 2962 SPY


posible caso: 3035 SPY
posible caso: 3130 SPY
posible caso: 3166 SPY


posible caso: 3396 SPY
posible caso: 3811 META
posible caso: 3859 META
posible caso: 4020 META


...

posible caso: 106041 GOOG


posible caso: 109479 APP
posible caso: 109507 APP


posible caso: 109539 APP
posible caso: 109559 APP


posible caso: 109640 APP
posible caso: 109690 APP


posible caso: 110001 APP
posible caso: 110014 APP


In [19]:
df_casos[(df_casos['caso']==1) & (df_casos['ind_posicion']==0)]

,Close,High,Low,Open,Volume,datetime,companyName,SMA20,SMA40,tipo_vela,...,bearishSlope,ind_posicion,promVol,sl_highs,r_sq_h,isBreakOut,isBreakOutIni,breakpointpos,isBreakOutFinal,caso
1954,538.340027,543.309998,538.130005,543.130005,10394986,2024-07-25 19:30:00+00:00,SPY,546.736862,549.418312,None,...,1.0,0,6.169648e+06,-0.731436,0.980411,NaN,NaN,NaN,NaN,1
4020,293.920013,298.345001,293.269989,295.700012,5566544,2023-09-21 13:30:00+00:00,META,302.620804,304.114916,None,...,1.0,0,2.416075e+06,-1.026390,0.815050,NaN,NaN,NaN,NaN,1
8196,183.315002,184.169998,180.050003,180.449997,35096943,2024-02-02 14:30:00+00:00,AAPL,186.268195,189.311170,None,...,1.0,0,6.687172e+06,-0.298133,0.893876,NaN,NaN,NaN,NaN,1
12610,158.184998,162.070007,151.610001,154.154999,30252093,2024-08-05 13:30:00+00:00,AMZN,177.430126,180.219506,None,...,1.0,0,6.276090e+06,-2.432384,0.970703,NaN,NaN,NaN,NaN,1
15646,543.989990,552.989990,542.010010,550.000000,2451675,2024-04-22 13:30:00+00:00,NFLX,590.059094,603.313652,None,...,1.0,0,5.730181e+05,-5.381037,0.987398,NaN,NaN,NaN,NaN,1
19028,101.570000,103.410004,100.300003,103.410004,960660,2024-03-20 13:30:00+00:00,MRNA,104.184750,105.277665,None,...,1.0,0,4.914014e+05,-0.259217,0.853454,NaN,NaN,NaN,NaN,1
21283,241.020004,244.949997,240.699997,244.710007,16072466,2023-06-26 19:30:00+00:00,TSLA,255.265883,259.907268,None,...,1.0,0,2.377124e+07,-1.816297,0.971654,NaN,NaN,NaN,NaN,1
48129,114.209999,115.970001,114.040001,115.839996,2644808,2024-09-03 13:30:00+00:00,XOM,117.196135,117.404562,None,...,1.0,0,1.274314e+06,-0.263936,0.625112,NaN,NaN,NaN,NaN,1
51482,458.250000,463.899994,458.049988,463.690002,7536069,2024-07-25 19:30:00+00:00,QQQ,469.837804,474.335101,None,...,1.0,0,5.513133e+06,-1.115789,0.978981,NaN,NaN,NaN,NaN,1
53186,328.815002,331.450012,327.000000,331.059998,6911006,2023-07-11 13:30:00+00:00,MSFT,336.380492,337.808375,None,...,1.0,0,2.722456e+06,-0.959354,0.997948,NaN,NaN,NaN,NaN,1


In [20]:
df_casos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17898 entries, 1879 to 110045
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   Close            17898 non-null  float64            
 1   High             17898 non-null  float64            
 2   Low              17898 non-null  float64            
 3   Open             17898 non-null  float64            
 4   Volume           17898 non-null  int64              
 5   datetime         17898 non-null  datetime64[ns, UTC]
 6   companyName      17898 non-null  object             
 7   SMA20            17879 non-null  float64            
 8   SMA40            17859 non-null  float64            
 9   tipo_vela        2565 non-null   object             
 10  pivotHigh        563 non-null    float64            
 11  pivotLow         576 non-null    float64            
 12  isPivot          1139 non-null   float64            
 13  dif_pivot        

In [21]:
#descarga de informacion, estrategia: RUPTURA DEL CANAL BAJISTA
path='data/cncf_h.txt'
# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df_casos.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)

In [22]:
df_casos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17898 entries, 1879 to 110045
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   Close            17898 non-null  float64            
 1   High             17898 non-null  float64            
 2   Low              17898 non-null  float64            
 3   Open             17898 non-null  float64            
 4   Volume           17898 non-null  int64              
 5   datetime         17898 non-null  datetime64[ns, UTC]
 6   companyName      17898 non-null  object             
 7   SMA20            17879 non-null  float64            
 8   SMA40            17859 non-null  float64            
 9   tipo_vela        2565 non-null   object             
 10  pivotHigh        563 non-null    float64            
 11  pivotLow         576 non-null    float64            
 12  isPivot          1139 non-null   float64            
 13  dif_pivot        

### BACKTESTING
##### 
Usando paquete backtesting.py

In [23]:
df_casos2 = df_casos.query("isBreakOutIni==1 or isBreakOutFinal==1").copy()

In [24]:
# =============== BACKTESTING ================= #
# obteniendo dataframe backtesting
df_backTesting = pd.merge(df, df_casos2[['datetime','companyName', 'isBreakOutIni','isBreakOutFinal','caso']], on = ['companyName','datetime'], how='left')
# datos de trade se asigna una vela anterior debido a que backtesting.py toma entra como la siguiente vela
df_backTesting['isBreakOutIni'] = df_backTesting['isBreakOutIni'].shift(-1)
df_backTesting['isBreakOutFinal'] = df_backTesting['isBreakOutFinal'].shift(-1)
#modificamos indice con el compo datetime
df_backTesting.set_index('datetime', inplace=True)

In [25]:
#Strategia
class strategyRupturaCanal(Strategy):    
    def init(self):        
        self.breakout_entry = self.I(lambda x: x, self.data.isBreakOutIni) #Indica la señal entrada al backtesting
        self.breakout_exit = self.I(lambda x: x, self.data.isBreakOutFinal) #Indica la señal salida al backtesting

    def next(self):       
        #print(self.data)        
        if self.breakout_entry[-1] == 1:
            self.buy()  #COMPRA
        elif self.breakout_exit[-1] == 1 and self.position:
            self.position.close() #CIERRE

In [26]:
# Por cada TIcket
my_stats = {}
estadisticas = pd.DataFrame()          # lista para ticker
tradesprev = pd.DataFrame()     # lista para cada caso por ticker
for ticker in df_backTesting['companyName'].unique():
    ticker_data = df_backTesting[df_backTesting['companyName'] == ticker].copy() #Filtro por cada ticker
    bt = Backtest(ticker_data, strategyRupturaCanal, cash=10_000)
    my_stats = bt.run()
    listTrades = my_stats['_trades']
    tradesdf = pd.DataFrame(listTrades) #trades
    tradesdf['Ticker'] = ticker
    stats_filter={
        'Ticker':ticker,
        'EntryTime':my_stats['Start'],
        'ExitTime':my_stats['End'],
        'Return [%]': my_stats['Return [%]'],
        'CAGR [%]': my_stats['Return (Ann.) [%]'],
        'Sharpe Ratio': my_stats['Sharpe Ratio'],
        'Max. Drawdown [%]': my_stats['Max. Drawdown [%]'],
        'Win Rate [%]': my_stats['Win Rate [%]'],
        '# Trades': my_stats['# Trades'],
        'Expectancy [%]': my_stats['Expectancy [%]'],
        'Profit Factor': my_stats['Profit Factor'],
        'Duration': my_stats['Duration'],
        'Avg. Trade [%]':my_stats['Avg. Trade [%]'],
        'Max. Trade Duration':my_stats['Max. Trade Duration'],
        'Avg. Trade Duration':my_stats['Avg. Trade Duration']
    }
    stat=pd.DataFrame([stats_filter])
    if tradesprev.shape[0]==0:
        tradesprev = tradesdf.copy()
        estadisticas=stat.copy()
    else:
        tradesprev = pd.concat([tradesprev, tradesdf],ignore_index=True)
        estadisticas=pd.concat([estadisticas, stat],ignore_index=True)
    # Mostramos
    #bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")

Backtest.run:   0%|          | 0/1583 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3016 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/2396 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/1524 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/2027 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/3535 [00:00<?, ?bar/s]

/tmp/ipykernel_2139/2801041912.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tradesprev = pd.concat([tradesprev, tradesdf],ignore_index=True)
/tmp/ipykernel_2139/2801041912.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estadisticas=pd.concat([estadisticas, stat],ignore_index=True)


Backtest.run:   0%|          | 0/496 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3535 [00:00<?, ?bar/s]

/tmp/ipykernel_2139/2801041912.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tradesprev = pd.concat([tradesprev, tradesdf],ignore_index=True)
/tmp/ipykernel_2139/2801041912.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estadisticas=pd.concat([estadisticas, stat],ignore_index=True)


In [27]:
ini = df_casos2[(df_casos2['isBreakOutIni']==1)].copy()
df_casos3_ini=ini.rename(columns={'Open': 'EntryPrice', 'datetime': 'EntryTime', 'companyName': 'Ticker'})[['EntryPrice','Ticker','caso','EntryTime']]
final = df_casos2[(df_casos2['isBreakOutFinal']==1)].copy()
df_casos3_final=final.rename(columns={'Open': 'ExitPrice', 'datetime': 'ExitTime', 'companyName': 'Ticker'})[['ExitPrice','Ticker','caso','ExitTime']]

In [28]:
df_casos3=df_casos3_ini.merge(df_casos3_final,on=['Ticker','caso'],how='outer')

In [29]:
trades = pd.merge(df_casos3, tradesprev, on = ['EntryTime','Ticker'], how='left')

In [30]:
def exitPrice(x):
    if pd.isna(x['ExitPrice_x']):
        return x['ExitPrice_y']
    else:
        return x['ExitPrice_x']
trades['ExitPrice'] = trades.apply(lambda row: exitPrice(row), axis=1)

In [31]:
trades = trades.rename(columns={'EntryPrice_x': 'EntryPrice'})
trades = trades.drop(['EntryPrice_y','ExitPrice_x', 'ExitPrice_y'], axis=1)

In [32]:
def exitTime(x):
    if pd.isna(x['ExitTime_x']):
        return x['ExitTime_y']
    else:
        return x['ExitTime_x']
trades['ExitTime'] = trades.apply(lambda row: exitTime(row), axis=1)

In [33]:
def duration(x):
    if pd.isna(x['Duration']):
        if pd.isna(x['ExitTime']):
            return pd.NaT
        else:
            return x['ExitTime']-x['EntryTime']
    else:
        return x['Duration']
trades['Duration'] = trades.apply(lambda row: duration(row), axis=1)

In [34]:
trades = trades.drop(['ExitTime_x', 'ExitTime_y'], axis=1)

In [35]:
trades=trades[['Ticker','EntryTime','ExitTime','EntryPrice','ExitPrice','Duration','Size','EntryBar','ExitBar','ReturnPct','caso']]

In [36]:
for item in estadisticas['Ticker'].unique():
    count = trades[trades["Ticker"] == item].shape[0]
    estadisticas.loc[estadisticas['Ticker'] == item, '# Trades'] = count

In [37]:
#path = r'D:\PRUEBAS'
path = 'data/backtesting'

os.makedirs(path,exist_ok=True)

path_trades = os.path.join(path, 'trades_cncf.txt')
path_estadisticas = os.path.join(path, 'estadisticas_cncf.txt')

for file_path in [path_trades, path_estadisticas]:
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"File does not exist: {file_path}. It will be created.")

trades.to_csv(path_trades, header=True, index=False, sep='\t', mode='w')
estadisticas.to_csv(path_estadisticas, header=True, index=False, sep='\t', mode='w')